In [1]:
!pip install pybacktestchain


In [2]:
from pybacktestchain.data_module import FirstTwoMoments
from pybacktestchain.broker import Backtest, StopLoss
from pybacktestchain.blockchain import load_blockchain
from datetime import datetime

# Set verbosity for logging
verbose = False  # Set to True to enable logging, or False to suppress it

backtest = Backtest(
    initial_date=datetime(2019, 1, 1),
    final_date=datetime(2020, 1, 1),
    information_class=FirstTwoMoments,
    risk_model=StopLoss,
    name_blockchain='backtest',
    verbose=verbose
)

backtest.run_backtest()

block_chain = load_blockchain('backtest')
print(str(block_chain))
# check if the blockchain is valid
print(block_chain.is_valid())

INFO:root:Blockchain with name backtest initialized and stored in the blockchain folder.
INFO:root:Running backtest from 2019-01-01 00:00:00 to 2020-01-01 00:00:00.
INFO:root:Retrieving price data for universe
c:\Users\ASUS\anaconda3\Lib\site-packages\pandas\core\frame.py:11211: RuntimeWarning: Degrees of freedom <= 0 for slice
  base_cov = np.cov(mat.T, ddof=ddof)
c:\Users\ASUS\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
c:\Users\ASUS\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
INFO:root:-----------------------------------
INFO:root:Rebalancing portfolio at 2019-01-31 00:00:00
c:\Users\ASUS\anaconda3\Lib\site-packages\pybacktestchain\broker.py:115: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer

--------------------------------------------------------------------------------
Block 0
--------------------------------------------------------------------------------
Backtest: Genesis Block
Timestamp: 1735640890.7700677
Hash: 5e687161c257fea2c315496aee5091b8aacc76e9d560f933a574a6625b31a713
Previous Hash: 0
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Block 1
--------------------------------------------------------------------------------
Backtest: AmberWolfBlacksmith
Timestamp: 1735640902.6022513
Hash: 238dd8c6591c8af3b5708ef655fb38853213ba7293fdd440e9a627fcef88d5d3
Previous Hash: 5e687161c257fea2c315496aee5091b8aacc76e9d560f933a574a6625b31a713
--------------------------------------------------------------------------------

True


In [3]:
import hashlib
import time
from dataclasses import dataclass, field

@dataclass
class Block:
    data: str
    previous_hash: str = ''
    timestamp: float = field(default_factory=time.time)
    hash: str = field(init=False)
    rewards: int = 0  # Reward for the block

    def __post_init__(self):
        self.hash = self.calculate_hash

    @property
    def calculate_hash(self):
        return hashlib.sha256(
            (str(self.timestamp) + self.data + self.previous_hash).encode()
        ).hexdigest()


@dataclass
class Blockchain:
    chain: list = field(default_factory=list)

    def __post_init__(self):
        self.chain.append(self.create_genesis_block())

    def create_genesis_block(self):
        return Block('Genesis Block', '0')

    def add_block(self, data: str, rewards: int):
        previous_block = self.chain[-1]
        new_block = Block(data, previous_block.hash, rewards=rewards)
        self.chain.append(new_block)

    def is_chain_valid(self):
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            previous_block = self.chain[i - 1]
            if current_block.previous_hash != previous_block.hash:
                return False
        return True


In [4]:
pragma solidity ^0.8.0;

contract RewardToken {
    string public name = "Reward Token";
    string public symbol = "RWD";
    uint8 public decimals = 18;
    uint256 public totalSupply;

    mapping(address => uint256) public balanceOf;

    event Transfer(address indexed from, address indexed to, uint256 value);

    constructor(uint256 initialSupply) {
        totalSupply = initialSupply * 10 ** uint256(decimals);
        balanceOf[msg.sender] = totalSupply; // Assign all tokens to the creator
    }

    function transfer(address to, uint256 value) public returns (bool success) {
        require(balanceOf[msg.sender] >= value, "Insufficient balance");
        balanceOf[msg.sender] -= value;
        balanceOf[to] += value;
        emit Transfer(msg.sender, to, value);
        return true;
    }
}


SyntaxError: invalid syntax (72485702.py, line 1)